In [27]:
field_size = 5 # 5x5, always square field
user_record = [] # list of lists for tracking the turns
win_line = [] # list of lists, win conditions that are still possible

In [57]:
# display function
def build_empty_field(field_size):
      row_up = "┌ - " + "- - " * (field_size - 2) + "- - ┐" + "\n" \
            + "|   " + "|   " * (field_size - 2) + "|   |" + "\n" \
            + "| - " + "+ - " * (field_size - 2) + "+ - |" + "\n" 
      row_mid = \
            "|   " + "|   " * (field_size - 2) + "|   |" + "\n" \
            + "| - " + "+ - " * (field_size - 2) + "+ - |" + "\n" 
      row_bot = \
            "|   " + "|   " * (field_size - 2) + "|   |" + "\n" \
            + "└ - " + "- - " * (field_size - 2) + "- - ┘" + "\n" 

      field = row_up + row_mid*(field_size-2) + row_bot
      return field

field = build_empty_field(6)        
print(field)
# print(middle_row)

┌ - - - - - - - - - - - ┐
|   |   |   |   |   |   |
| - + - + - + - + - + - |
|   |   |   |   |   |   |
| - + - + - + - + - + - |
|   |   |   |   |   |   |
| - + - + - + - + - + - |
|   |   |   |   |   |   |
| - + - + - + - + - + - |
|   |   |   |   |   |   |
| - + - + - + - + - + - |
|   |   |   |   |   |   |
└ - - - - - - - - - - - ┘



In [2]:
## GLOBAL VARIABLES
field_size = 3
field_pos_label = []         # position label for whole play field
available_pos_label = []     # position label for empty spot
user_record = [[], []]       # record input for user1/2
win_line = []                # lines on play field for winning
def init_field(field_size_n, user_history=None):
    """ Initialize the play field
    Args:
        field_size_n (int): a general play field nxn
        user_history (list): list of two user position history:
              [[usr1_pos1, usr2_pos2, usr1_pos3], [usr2_pos1, usr2_pos2]]].
              Defaults to None.
    """
    field_size = field_size_n
    field_pos_label = list(range(1, 1+field_size**2))
    if user_history == None:
        user_record = [[], []]
        available_pos_label = field_pos_label
    else:
        user_record = user_history
        ## check consistence
        if len(user_record[0]) != len(set(user_record[0])):
               print("warning: duplicate elements within user_history[0]")
               exit(0)
        if len(user_record[1]) != len(set(user_record[1])):
               print("warning: duplicate elements within user_history[1]")
               exit(0)
        if user_record[0] not in field_pos_label:
               print("warning: label overflow in user_history[0]")
               exit(0)
        if user_record[1] not in field_pos_label:
               print("warning: label overflow in user_history[1]")
               exit(0)
        available_pos_label = list(set(field_pos_label)
                                   - set(user_record[0])
                                   - set(user_record[1]))
    win_line = []
    tmp = list(range(field_size))
    for i in range(field_size):
        win_line.append([1+x+i*field_size for x in tmp])
        win_line.append([1+x*field_size+i for x in tmp])
    win_line.append([1+x*field_size+x for x in tmp])
    win_line.append([(1+x)*field_size-x for x in tmp])
    return user_record, win_line, available_pos_label

In [5]:
def validate(available_pos_label):
    avaiable_pos_label = set(field_pos_label) - set(user_record[0]) \
                         - set(user_record[1])
    if len(avaiable_pos_label) == 0:
          print('warning: no available position to continue game!')
          print('game over!')
          exit(0)
    turn_valid = False
    while not turn_valid:
        if len(user_record[0]) == len(user_record[1]):
        # means it is the first player's turn
            turn = int(input("Player-1 (X): Please choose one available position label"))
            if turn not in available_pos_label:
                 turn_valid = True

                 user_record[0].append(turn)
        # means it is the second player's turn
            turn = int(input("Player-2 (O): Please choose one available position label"))
            if turn not in available_pos_label:
                 turn_valid = True
                 user_record[0].append(turn)

    pass

In [4]:
init_field(4, None)

([[], []],
 [[1, 2, 3, 4],
  [1, 5, 9, 13],
  [5, 6, 7, 8],
  [2, 6, 10, 14],
  [9, 10, 11, 12],
  [3, 7, 11, 15],
  [13, 14, 15, 16],
  [4, 8, 12, 16],
  [1, 6, 11, 16],
  [4, 7, 10, 13]],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])